In [40]:
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
%matplotlib inline

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2.11.0
The following GPU devices are available: 


In [ ]:
def display_image(image):
    fig = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.imshow(image)

In [ ]:
def resize_image(path, new_width=256, new_height=256, display=False):
    pil_image = image.open(path)
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    pil_image_rgb = pil_image.convert("RGB")
    return pil_image_rgb

In [69]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
    """Adds a bounding box to an image."""
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),(left, top)],
              width=thickness,
              fill=color)

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin), (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str,
                  fill="black",
                  font=font)
        text_bottom -= text_height - 2 * margin

In [71]:
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())
    font = ImageFont.load_default()
    for i in range(min(boxes.shape[0], max_boxes)):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            #display_str = "{}: {}%".format(class_names[i].decode("ascii"), int(100 * scores[i]))
            
            color = colors[hash(class_names[i]) % len(colors)]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
            draw_bounding_box_on_image(
              image_pil,
              ymin,
              xmin,
              ymax,
              xmax,
              color,
              font,
              display_str_list=[display_str])
            np.copyto(image, np.array(image_pil))
    return image

In [47]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img

In [93]:
import os
path = ./imagesimgs = [path+img for img in os.listdir(path) if os.path.isfile(os.path.join(path, img))]

In [46]:
#load pretrained model
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]
detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [124]:
def run_detector(detector, path, display=False):
    img = load_img(path)

    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    start_time = time.time()
    result = detector(converted_img)
    end_time = time.time()

    result = {key:value.numpy() for key,value in result.items()}

    image_with_boxes = draw_boxes(img.numpy(), result["detection_boxes"],
                                  result["detection_class_entities"],
                                  result["detection_scores"])
    if display==True:
        display_image(image_with_boxes)
        
    return {"image_path": path, "inference_time": end_time-start_time, "output": result}

In [125]:
test = run_detector(detector, imgs[0], display=False)

In [126]:
test

{'image_path': 'C:\\Users\\j.driscoll\\downloads\\jiasc_test\\0.JPG',
 'inference_time': 18.255093812942505,
 'output': {'detection_class_names': array([b'/m/0k4j', b'/m/083wq', b'/m/01prls', b'/m/083wq', b'/m/07yv9',
         b'/m/083wq', b'/m/07yv9', b'/m/083wq', b'/m/07j7r', b'/m/07j7r',
         b'/m/01g317', b'/m/07cmd', b'/m/083wq', b'/m/01prls', b'/m/0d4v4',
         b'/m/083wq', b'/m/07yv9', b'/m/083wq', b'/m/0h9mv', b'/m/083wq',
         b'/m/01prls', b'/m/083wq', b'/m/0k4j', b'/m/0d4v4', b'/m/0h9mv',
         b'/m/0d4v4', b'/m/03jm5', b'/m/0cgh4', b'/m/07j7r', b'/m/083wq',
         b'/m/08dz3q', b'/m/01g317', b'/m/0h9mv', b'/m/0h9mv', b'/m/0cgh4',
         b'/m/07j7r', b'/m/01g317', b'/m/09j2d', b'/m/083wq', b'/m/07j7r',
         b'/m/0138tl', b'/m/083wq', b'/m/01prls', b'/m/0h9mv', b'/m/0cgh4',
         b'/m/01prls', b'/m/083wq', b'/m/07j7r', b'/m/07yv9', b'/m/0h9mv',
         b'/m/0h9mv', b'/m/083wq', b'/m/07cmd', b'/m/0cgh4', b'/m/05s2s',
         b'/m/083wq', b'/m/0138tl'

In [ ]:
# run model on all sample images
results = []
for img in imgs:
    output = run_detector(detector, img)
    results.append(output)

In [138]:
#write to file

import pickle

with open(f"{path}\\results\\fasterrcnn.pkl", 'wb') as f:
    pickle.dump(test, f)

In [139]:
with open(f"{path}\\results\\test.pkl", 'rb') as f:
    data = pickle.load(f)